# PRACTICA 1

In [14]:
# Abrir datasets
import pandas as pd

df_white = pd.read_csv("data/winequality-white.csv", sep=';')

# Eliminación de duplicados, ignorado quality
cols = df_white.columns.drop('quality')
df_white = df_white.drop_duplicates(subset=cols)

In [15]:
print("lucia intenta ter unha celda pa traballar e facer un maldito commit")

lucia intenta ter unha celda pa traballar e facer un maldito commit


In [ ]:
df_white = pd.read_csv("data/winequality-white.csv", sep=';')
def clasificar_vino(valor): 
    if valor < 7: 
        return "baja calidad" 
    elif valor >= 7: 
        return "alta calidad" 
df['calidad'] = df['quality'].apply(clasificar_vino) 
df = df.drop(columns=['quality']) 
# Guardar el CSV mod 
df.to_csv("data\winequality-white-procesado.csv", index=False, sep=";")

<>:10: SyntaxWarning: invalid escape sequence '\w'
<>:10: SyntaxWarning: invalid escape sequence '\w'
C:\Users\User\AppData\Local\Temp\ipykernel_24564\3509338710.py:10: SyntaxWarning: invalid escape sequence '\w'
  df.to_csv("data\winequality-white-procesado.csv", index=False, sep=";")
